# 네이버 영화 리뷰 감정 분석을 위한 데이터 파이프라인

## 개요
이 노트북에서는 네이버 영화 리뷰 데이터(NSMC)를 사용하여 감정 분석을 위한 완전한 데이터 파이프라인을 구축합니다.

### 주요 학습 내용:
1. **Korpora 라이브러리**를 통한 NSMC 데이터셋 다운로드
2. **데이터 구조화**: train/validation/test 분할 및 파일 시스템 구조 생성
3. **TensorFlow Dataset API**를 활용한 대용량 데이터 처리
4. **한글 텍스트 전처리**: KoNLPy + TensorFlow 통합
5. **벡터화 파이프라인**: 실시간 데이터 변환
6. **성능 최적화**: 캐싱, 프리페칭, 병렬 처리

## 1. 라이브러리 및 환경 설정

### 1.1 필수 라이브러리 설치
```bash
# Korpora 라이브러리 설치 (conda로 설치가 안되므로 pip 사용)
pip install Korpora
```

In [ ]:
# 필요한 라이브러리 import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from konlpy.tag import Okt
import re
import os
import pathlib
import shutil
import random
import numpy as np
from keras import layers

print(f"TensorFlow 버전: {tf.__version__}")
print(f"Python 버전: {tf.version.VERSION}")

# GPU 사용 가능 여부 확인
print(f"GPU 사용 가능: {tf.config.list_physical_devices('GPU')}")

## 2. NSMC 데이터셋 다운로드 및 탐색

### 2.1 Korpora를 통한 데이터 다운로드

In [ ]:
import os
from Korpora import Korpora

# 데이터 저장 경로 설정
data_dir = "../../data"
os.makedirs(data_dir, exist_ok=True)

# NSMC(Naver Sentiment Movie Corpus) 데이터셋 다운로드
# 기본 경로 대신 지정한 경로에 저장
print("NSMC 데이터셋 다운로드 중...")
print(f"저장 경로: {os.path.abspath(data_dir)}")

# root_dir 파라미터를 사용하여 저장 위치 지정
Korpora.fetch("nsmc", root_dir=data_dir)
print("다운로드 완료!")

### 2.2 데이터셋 로드 및 구조 확인

In [ ]:
# 데이터셋 로드
corpus = Korpora.load("nsmc")

# 데이터 구조 탐색
print("=== NSMC 데이터셋 정보 ===")
print(f"훈련 데이터 크기: {len(corpus.train.texts)}")
print(f"테스트 데이터 크기: {len(corpus.test.texts)}")
print()

print("=== 샘플 데이터 (첫 3개) ===")
for i in range(3):
    print(f"샘플 {i+1}:")
    print(f"  텍스트: {corpus.train.texts[i]}")
    print(f"  라벨: {corpus.train.labels[i]} ({'긍정' if corpus.train.labels[i] == 1 else '부정'})")  # 정수 1과 비교
    print()

### 2.3 라벨링 검증 및 원본 데이터 확인


In [ ]:
# 원본 파일을 직접 읽어서 확인해보자
import pandas as pd

print("=== 원본 파일 직접 확인 ===")
try:
    # 원본 TSV 파일 직접 읽기
    train_file_path = os.path.join(data_dir, "nsmc", "ratings_train.txt")
    
    # 첫 몇 줄을 직접 읽어보기
    with open(train_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()[:10]  # 첫 10줄
    
    print("원본 파일 첫 10줄:")
    for i, line in enumerate(lines):
        print(f"{i}: {line.strip()}")
    
    print("\n" + "="*50)
    
    # pandas로 정확히 파싱해보기
    df_train = pd.read_csv(train_file_path, sep='\t', encoding='utf-8')
    print(f"pandas로 읽은 데이터:")
    print(f"컬럼명: {list(df_train.columns)}")
    print(f"데이터 형태: {df_train.shape}")
    print("\n첫 5개 샘플:")
    print(df_train.head())
    
    print(f"\n라벨 분포:")
    print(df_train['label'].value_counts().sort_index())
    
    # Korpora 데이터와 비교
    print(f"\n=== Korpora vs 원본 데이터 비교 ===")
    print("첫 5개 샘플 비교:")
    for i in range(5):
        korpora_text = corpus.train.texts[i]
        korpora_label = corpus.train.labels[i]
        pandas_text = df_train.iloc[i]['document']
        pandas_label = df_train.iloc[i]['label']
        
        print(f"\n샘플 {i+1}:")
        print(f"  Korpora: '{korpora_text}' | 라벨: {korpora_label}")
        print(f"  원본:    '{pandas_text}' | 라벨: {pandas_label}")
        print(f"  매치: {korpora_text == pandas_text and korpora_label == pandas_label}")

except Exception as e:
    print(f"원본 파일 읽기 실패: {e}")

In [ ]:
# 라벨링 문제 분석: 샘플들을 더 자세히 살펴보자
print("=== 의심스러운 라벨링 샘플 분석 ===")

# 명백히 긍정적인 표현들을 가진 부정 라벨 샘플들 찾기
positive_words = ['좋', '재미', '추천', '최고', '훌륭', '감동', '멋진', '완벽']
negative_words = ['짜증', '별로', '최악', '실망', '지루', '재미없', '아깝']

print("부정 라벨(0)인데 긍정 단어가 포함된 샘플들:")
positive_in_negative = []
for i in range(min(1000, len(corpus.train.texts))):  # 첫 1000개만 확인
    text = corpus.train.texts[i]
    label = corpus.train.labels[i]
    
    if label == 0:  # 부정 라벨
        if any(pos_word in text for pos_word in positive_words):
            positive_in_negative.append((i, text, label))
            if len(positive_in_negative) <= 5:  # 처음 5개만 출력
                print(f"  {i}: [{label}] {text}")

print(f"\n총 {len(positive_in_negative)}개 발견")

print("\n" + "="*50)
print("긍정 라벨(1)인데 부정 단어가 포함된 샘플들:")
negative_in_positive = []
for i in range(min(1000, len(corpus.train.texts))):
    text = corpus.train.texts[i]
    label = corpus.train.labels[i]
    
    if label == 1:  # 긍정 라벨
        if any(neg_word in text for neg_word in negative_words):
            negative_in_positive.append((i, text, label))
            if len(negative_in_positive) <= 5:  # 처음 5개만 출력
                print(f"  {i}: [{label}] {text}")

print(f"\n총 {len(negative_in_positive)}개 발견")

# 전체 라벨 분포 다시 확인
print("\n" + "="*50)
print("=== 전체 라벨 분포 확인 ===")
pos_count = sum(1 for label in corpus.train.labels if label == 1)
neg_count = sum(1 for label in corpus.train.labels if label == 0)
total = len(corpus.train.labels)

print(f"긍정(1): {pos_count:,}개 ({pos_count/total*100:.1f}%)")
print(f"부정(0): {neg_count:,}개 ({neg_count/total*100:.1f}%)")
print(f"총 샘플: {total:,}개")

# 데이터 타입과 고유값 확인
print(f"\n라벨 데이터 타입: {type(corpus.train.labels[0])}")
print(f"라벨 고유값: {set(corpus.train.labels)}")

print("\n=== 결론 ===")
print("NSMC 데이터셋은 실제로 노이즈가 있는 데이터셋으로 알려져 있습니다.")
print("일부 라벨링 오류가 존재할 수 있으며, 이는 실제 데이터에서 흔히 발생하는 문제입니다.")
print("모델 훈련 시에는 이러한 노이즈를 고려하여 진행해야 합니다.")

## 3. 데이터 구조화 및 파일 시스템 준비

TensorFlow의 `text_dataset_from_directory`를 사용하기 위해 데이터를 디렉토리 구조로 정리합니다.

### 3.1 데이터 구조화 함수 정의

In [ ]:
def create_korean_dataset(base_dir=None, validation_size=1000):
    """
    NSMC 데이터를 TensorFlow Dataset 형식의 디렉토리 구조로 변환
    
    구조:
    ../../data/korean_imdb/
    ├── train/
    │   ├── pos/  (긍정 리뷰들)
    │   └── neg/  (부정 리뷰들)
    ├── val/
    │   ├── pos/
    │   └── neg/
    └── test/
        ├── pos/
        └── neg/
    
    Args:
        base_dir (str): 디렉토리 이름 (기본값: None이면 data_dir/korean_imdb 사용)
        validation_size (int): 검증 데이터 크기 (각 클래스별)
    
    Returns:
        str: 생성된 데이터 디렉토리 경로
    """
    
    if base_dir is None:
        base_dir = os.path.join(data_dir, "korean_imdb")
    
    print(f"데이터 구조화 시작: {base_dir}")
    
    # 기존 디렉토리 삭제 (있다면)
    if os.path.exists(base_dir):
        try:
            shutil.rmtree(base_dir)
            print(f"기존 디렉토리 삭제: {base_dir}")
        except OSError as e:
            print(f"디렉토리 삭제 오류: {e}")
    
    # 디렉토리 구조 생성
    directories = [
        os.path.join(base_dir, "train", "pos"),
        os.path.join(base_dir, "train", "neg"),
        os.path.join(base_dir, "val", "pos"),
        os.path.join(base_dir, "val", "neg"),
        os.path.join(base_dir, "test", "pos"),
        os.path.join(base_dir, "test", "neg")
    ]
    
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    
    print("디렉토리 구조 생성 완료")
    
    return base_dir

### 3.2 데이터 분할 및 파일 저장

In [ ]:
def save_data_to_files(corpus, base_dir, validation_size=1000):
    """
    corpus 데이터를 파일로 저장
    """
    
    # 훈련 데이터 라벨별 분리
    print("훈련 데이터 분리 중...")
    pos_train_texts = []
    neg_train_texts = []
    
    for text, label in zip(corpus.train.texts, corpus.train.labels):
        if label == 1:  # 정수 1로 비교
            pos_train_texts.append(text)
        elif label == 0:  # 정수 0로 비교
            neg_train_texts.append(text)
    
    print(f"긍정 훈련 데이터: {len(pos_train_texts):,}개")
    print(f"부정 훈련 데이터: {len(neg_train_texts):,}개")
    
    # 테스트 데이터 라벨별 분리
    print("테스트 데이터 분리 중...")
    pos_test_texts = []
    neg_test_texts = []
    
    for text, label in zip(corpus.test.texts, corpus.test.labels):
        if label == 1:  # 정수 1로 비교
            pos_test_texts.append(text)
        elif label == 0:  # 정수 0로 비교
            neg_test_texts.append(text)
    
    print(f"긍정 테스트 데이터: {len(pos_test_texts):,}개")
    print(f"부정 테스트 데이터: {len(neg_test_texts):,}개")
    
    # 데이터가 충분한지 확인
    if len(pos_train_texts) < validation_size or len(neg_train_texts) < validation_size:
        print(f"경고: 검증 데이터 분할을 위한 충분한 데이터가 없습니다.")
        validation_size = min(len(pos_train_texts), len(neg_train_texts)) // 2
        print(f"검증 데이터 크기를 {validation_size}개로 조정합니다.")
    
    # 검증 데이터 분할 (훈련 데이터에서 일부 추출)
    print(f"검증 데이터 분할 중... (각 클래스별 {validation_size}개)")
    pos_val_texts = pos_train_texts[:validation_size] if pos_train_texts else []
    neg_val_texts = neg_train_texts[:validation_size] if neg_train_texts else []
    
    # 훈련 데이터에서 검증 데이터 제거
    pos_train_texts = pos_train_texts[validation_size:] if pos_train_texts else []
    neg_train_texts = neg_train_texts[validation_size:] if neg_train_texts else []
    
    print(f"최종 훈련 데이터 - 긍정: {len(pos_train_texts):,}개, 부정: {len(neg_train_texts):,}개")
    print(f"검증 데이터 - 긍정: {len(pos_val_texts):,}개, 부정: {len(neg_val_texts):,}개")
    
    # 파일 저장 함수
    def save_texts_to_files(texts, directory, prefix):
        os.makedirs(directory, exist_ok=True)  # 디렉토리가 없으면 생성
        for i, text in enumerate(texts):
            filename = f"{prefix}_{i}.txt"
            filepath = os.path.join(directory, filename)
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(text)
    
    # 훈련 데이터 저장
    print("훈련 데이터 파일 저장 중...")
    if pos_train_texts:
        save_texts_to_files(pos_train_texts, os.path.join(base_dir, "train", "pos"), "pos")
    if neg_train_texts:
        save_texts_to_files(neg_train_texts, os.path.join(base_dir, "train", "neg"), "neg")
    
    # 검증 데이터 저장
    print("검증 데이터 파일 저장 중...")
    if pos_val_texts:
        save_texts_to_files(pos_val_texts, os.path.join(base_dir, "val", "pos"), "pos")
    if neg_val_texts:
        save_texts_to_files(neg_val_texts, os.path.join(base_dir, "val", "neg"), "neg")
    
    # 테스트 데이터 저장
    print("테스트 데이터 파일 저장 중...")
    if pos_test_texts:
        save_texts_to_files(pos_test_texts, os.path.join(base_dir, "test", "pos"), "pos")
    if neg_test_texts:
        save_texts_to_files(neg_test_texts, os.path.join(base_dir, "test", "neg"), "neg")
    
    print("모든 데이터 파일 저장 완료!")
    return {
        'train_pos': len(pos_train_texts),
        'train_neg': len(neg_train_texts), 
        'val_pos': len(pos_val_texts),
        'val_neg': len(neg_val_texts),
        'test_pos': len(pos_test_texts),
        'test_neg': len(neg_test_texts)
    }

# 실행 (시간이 오래 걸릴 수 있습니다)
korean_data_dir = create_korean_dataset()
data_stats = save_data_to_files(corpus, korean_data_dir)

print(f"데이터 디렉토리: {korean_data_dir}")
print(f"데이터 통계: {data_stats}")

## 4. TensorFlow Dataset 로드

### 4.1 데이터셋 초기 로드

In [ ]:
# 배치 크기 설정
BATCH_SIZE = 32

print(f"배치 크기: {BATCH_SIZE}")
print("TensorFlow Dataset 로드 중...")

try:
    # text_dataset_from_directory를 사용한 데이터셋 로드
    # 오타 수정: dataset_from_directory -> text_dataset_from_directory
    train_ds_raw = keras.utils.text_dataset_from_directory(
        korean_data_dir + "/train", 
        batch_size=BATCH_SIZE, 
        label_mode="binary",
        validation_split=None
    )
    
    val_ds_raw = keras.utils.text_dataset_from_directory(
        korean_data_dir + "/val", 
        batch_size=BATCH_SIZE, 
        label_mode="binary",
        validation_split=None
    )
    
    test_ds_raw = keras.utils.text_dataset_from_directory(
        korean_data_dir + "/test", 
        batch_size=BATCH_SIZE, 
        label_mode="binary",
        validation_split=None
    )
    
    print("데이터셋 로드 완료!")
    
except Exception as e:
    print(f"데이터셋 로드 실패: {e}")
    print("먼저 데이터 구조화를 실행해주세요.")

### 4.2 로드된 데이터 확인

In [ ]:
# 데이터셋 구조 확인
print("=== 데이터셋 정보 ===")

def get_dataset_info(dataset, name):
    print(f"{name}:")
    print(f"  요소 스펙: {dataset.element_spec}")
    
    # 첫 번째 배치 가져오기
    for texts, labels in dataset.take(1):
        print(f"  배치 크기: {texts.shape[0]}")
        print(f"  텍스트 형태: {texts.shape}")
        print(f"  라벨 형태: {labels.shape}")
        print(f"  첫 번째 텍스트 샘플: {texts[0].numpy().decode('utf-8')[:100]}...")
        print(f"  첫 번째 라벨: {labels[0].numpy()}")
    print()

try:
    get_dataset_info(train_ds_raw, "훈련 데이터셋")
    get_dataset_info(val_ds_raw, "검증 데이터셋")
    get_dataset_info(test_ds_raw, "테스트 데이터셋")
except:
    print("데이터셋이 로드되지 않았습니다.")

## 5. 한국어 텍스트 전처리 파이프라인

### 5.1 텍스트 정제 함수

In [ ]:
# 형태소 분석기 초기화
okt = Okt()
print("KoNLPy Okt 형태소 분석기 초기화 완료")

def clean_text(text):
    """
    텍스트 정제 함수
    
    주의: TensorFlow에서 전달되는 텍스트는 bytes 형태로 인코딩되어 있으므로
    먼저 UTF-8로 디코딩해야 합니다.
    
    Args:
        text (bytes): TensorFlow에서 전달된 인코딩된 텍스트
    
    Returns:
        str: 정제된 텍스트
    """
    # TensorFlow tensor를 Python string으로 디코딩
    text = text.decode("utf-8")
    
    # 소문자 변환
    text = text.lower()
    
    # 한글, 영문, 숫자, 공백만 유지
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]", "", text)
    
    return text

# 테스트
test_text = "이 영화는 정말 좋았어요! 5점 만점에 5점★★★"
test_bytes = test_text.encode('utf-8')
cleaned = clean_text(test_bytes)
print(f"원본: {test_text}")
print(f"정제: {cleaned}")

### 5.2 배치 전처리 함수

In [ ]:
def python_korean_preprocess(text_tensor):
    """
    TensorFlow 텐서 배치에 대한 한국어 전처리
    
    이 함수는 tf.py_function을 통해 호출되며,
    TensorFlow 텐서를 받아서 처리한 후 다시 텐서로 반환합니다.
    
    Args:
        text_tensor (tf.Tensor): 텍스트 배치 텐서
    
    Returns:
        tf.Tensor: 전처리된 텍스트 배치 텐서
    """
    processed_texts = []
    
    # 배치의 각 텍스트에 대해 처리
    for text_bytes in text_tensor.numpy():
        # 1단계: 텍스트 정제
        cleaned_text = clean_text(text_bytes)
        
        # 2단계: 형태소 분석
        morphed_tokens = okt.morphs(cleaned_text)
        
        # 3단계: 토큰들을 공백으로 결합
        morphed_text = " ".join(morphed_tokens)
        
        processed_texts.append(morphed_text)
    
    # Python 리스트를 TensorFlow 텐서로 변환
    return tf.constant(processed_texts, dtype=tf.string)

print("배치 전처리 함수 정의 완료")

### 5.3 TensorFlow 통합 전처리 함수

In [ ]:
def tf_korean_preprocess_fn(texts, labels):
    """
    TensorFlow Dataset에서 사용할 전처리 함수
    
    tf.py_function을 사용하여 Python 함수를 TensorFlow 그래프에 통합합니다.
    이를 통해 KoNLPy와 같은 Python 라이브러리를 TensorFlow 파이프라인에서 사용할 수 있습니다.
    
    Args:
        texts (tf.Tensor): 텍스트 배치
        labels (tf.Tensor): 라벨 배치
    
    Returns:
        tuple: (전처리된 텍스트, 라벨)
    """
    # 오타 수정: tf.py_func -> tf.py_function
    processed_texts = tf.py_function(
        func=python_korean_preprocess,  # 실행할 Python 함수
        inp=[texts],                    # 입력 데이터
        Tout=tf.string                  # 출력 데이터 타입
    )
    
    # 명시적으로 shape 설정 (TensorFlow 요구사항)
    processed_texts.set_shape(texts.get_shape())
    
    return processed_texts, labels

print("TensorFlow 통합 전처리 함수 정의 완료")

## 6. 텍스트 벡터화 설정

### 6.1 TextVectorization 레이어 생성

In [ ]:
# 벡터화 파라미터 설정
MAX_TOKENS = 10000          # 어휘 사전 크기 (상위 10,000개 빈도 단어)
OUTPUT_SEQUENCE_LENGTH = 20 # 출력 시퀀스 길이 (패딩/자르기)

print(f"벡터화 설정:")
print(f"  최대 토큰 수: {MAX_TOKENS:,}")
print(f"  시퀀스 길이: {OUTPUT_SEQUENCE_LENGTH}")

# TextVectorization 레이어 생성
# 오타 수정: standarize -> standardize
vectorizer = TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode="int",                    # 정수 시퀀스 출력
    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
    standardize=None,                     # 외부에서 표준화 수행
    split="whitespace"                    # 공백 기준 토큰 분리
)

print("TextVectorization 레이어 생성 완료")

### 6.2 데이터 전처리 적용

In [ ]:
try:
    print("데이터 전처리 적용 중...")
    
    # 오타 수정: num_paralle_calls -> num_parallel_calls
    # 모든 데이터셋에 한국어 전처리 적용
    train_ds_processed = train_ds_raw.map(
        tf_korean_preprocess_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    val_ds_processed = val_ds_raw.map(
        tf_korean_preprocess_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    test_ds_processed = test_ds_raw.map(
        tf_korean_preprocess_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    print("전처리 완료!")
    
    # 전처리 결과 확인
    print("\n전처리 결과 샘플:")
    for texts, labels in train_ds_processed.take(1):
        print(f"원본 형태소 분석 결과: {texts[0].numpy().decode('utf-8')}")
        print(f"라벨: {labels[0].numpy()}")
        
except Exception as e:
    print(f"전처리 실패: {e}")
    print("데이터셋이 로드되지 않았거나 오류가 발생했습니다.")

## 7. 어휘 사전 구축 및 벡터화

### 7.1 어휘 사전 학습

In [ ]:
try:
    print("어휘 사전 학습 중...")
    
    # 훈련 데이터의 텍스트만 추출하여 어휘 사전 학습
    # lambda 함수를 사용해 텍스트(x)만 선택, 라벨(y)은 무시
    text_only_dataset = train_ds_processed.map(lambda x, y: x)
    
    # 어휘 사전 학습 (adapt)
    vectorizer.adapt(text_only_dataset)
    
    print("어휘 사전 학습 완료!")
    
    # 어휘 사전 정보 확인
    vocabulary = vectorizer.get_vocabulary()
    print(f"\n어휘 사전 크기: {len(vocabulary):,}")
    print("상위 20개 어휘:")
    for i, word in enumerate(vocabulary[:20]):
        print(f"  {i:2d}: '{word}'")
    
except Exception as e:
    print(f"어휘 사전 학습 실패: {e}")

### 7.2 벡터화 함수 및 적용

In [ ]:
def vectorize_text_fn(texts, labels):
    """
    텍스트 벡터화 함수
    
    Args:
        texts (tf.Tensor): 전처리된 텍스트 배치
        labels (tf.Tensor): 라벨 배치
    
    Returns:
        tuple: (벡터화된 텍스트, 라벨)
    """
    return vectorizer(texts), labels

try:
    print("텍스트 벡터화 적용 중...")
    
    # 벡터화 적용
    train_ds_vectorized = train_ds_processed.map(
        vectorize_text_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    val_ds_vectorized = val_ds_processed.map(
        vectorize_text_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    test_ds_vectorized = test_ds_processed.map(
        vectorize_text_fn, 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    print("벡터화 완료!")
    
    # 벡터화 결과 확인
    print("\n벡터화 결과 샘플:")
    for vectors, labels in train_ds_vectorized.take(1):
        print(f"벡터 형태: {vectors.shape}")
        print(f"첫 번째 벡터: {vectors[0].numpy()}")
        print(f"라벨: {labels[0].numpy()}")
        
except Exception as e:
    print(f"벡터화 실패: {e}")

## 8. 성능 최적화 적용

### 8.1 캐싱 및 프리페칭

In [ ]:
try:
    print("성능 최적화 적용 중...")
    
    # 캐싱과 프리페칭을 통한 성능 최적화
    # - cache(): 메모리에 데이터를 캐시하여 반복 접근 시 성능 향상
    # - prefetch(): 다음 배치를 미리 준비하여 GPU 유휴 시간 최소화
    
    train_ds_final = train_ds_vectorized.cache().prefetch(
        buffer_size=tf.data.AUTOTUNE
    )
    
    val_ds_final = val_ds_vectorized.cache().prefetch(
        buffer_size=tf.data.AUTOTUNE
    )
    
    test_ds_final = test_ds_vectorized.cache().prefetch(
        buffer_size=tf.data.AUTOTUNE
    )
    
    print("성능 최적화 완료!")
    
    print("\n=== 최종 데이터셋 정보 ===")
    print("모든 데이터셋이 다음 단계를 거쳐 처리되었습니다:")
    print("  1. 원본 텍스트 로드")
    print("  2. 한글 텍스트 정제 및 형태소 분석")
    print("  3. 텍스트 벡터화 (정수 시퀀스 변환)")
    print("  4. 캐싱 및 프리페칭 최적화")
    print("\n이제 딥러닝 모델 훈련에 사용할 준비가 완료되었습니다!")
    
except Exception as e:
    print(f"성능 최적화 실패: {e}")

## 9. 데이터 파이프라인 검증

### 9.1 최종 데이터 형태 확인

In [ ]:
try:
    print("=== 최종 데이터 파이프라인 검증 ===")
    
    def validate_dataset(dataset, name):
        print(f"\n{name} 검증:")
        
        batch_count = 0
        total_samples = 0
        
        for vectors, labels in dataset.take(3):  # 첫 3개 배치만 확인
            batch_count += 1
            batch_size = vectors.shape[0]
            total_samples += batch_size
            
            print(f"  배치 {batch_count}: {batch_size}개 샘플")
            print(f"    벡터 형태: {vectors.shape}")
            print(f"    라벨 형태: {labels.shape}")
            print(f"    벡터 데이터 타입: {vectors.dtype}")
            print(f"    라벨 데이터 타입: {labels.dtype}")
            
            # 샘플 벡터 확인
            sample_vector = vectors[0].numpy()
            non_zero_count = np.count_nonzero(sample_vector)
            print(f"    샘플 벡터 유효 토큰 수: {non_zero_count}/{len(sample_vector)}")
            print(f"    샘플 라벨: {labels[0].numpy()}")
        
        print(f"  확인한 총 샘플 수: {total_samples}")
    
    # 각 데이터셋 검증
    validate_dataset(train_ds_final, "훈련 데이터셋")
    validate_dataset(val_ds_final, "검증 데이터셋") 
    validate_dataset(test_ds_final, "테스트 데이터셋")
    
except Exception as e:
    print(f"데이터셋 검증 실패: {e}")

### 9.2 처리 과정 요약

In [ ]:
print("=== 한국어 텍스트 처리 파이프라인 요약 ===")
print()
print("🔍 1단계: 데이터 수집")
print("   - Korpora를 통한 NSMC 데이터셋 다운로드")
print("   - 네이버 영화 리뷰 150,000개 + 테스트 50,000개")
print()
print("📁 2단계: 데이터 구조화")
print("   - train/val/test 분할")
print("   - pos(긍정)/neg(부정) 디렉토리 구조 생성")
print("   - 각 리뷰를 개별 텍스트 파일로 저장")
print()
print("📊 3단계: TensorFlow Dataset 로드")
print("   - text_dataset_from_directory 사용")
print("   - 배치 단위 데이터 로딩")
print("   - 이진 분류용 라벨 설정")
print()
print("🔧 4단계: 텍스트 전처리")
print("   - UTF-8 디코딩 (TensorFlow tensor → Python string)")
print("   - 특수문자 제거, 소문자 변환")
print("   - KoNLPy Okt를 통한 형태소 분석")
print("   - tf.py_function으로 TensorFlow와 통합")
print()
print("🔢 5단계: 텍스트 벡터화")
print("   - TextVectorization으로 어휘 사전 구축")
print("   - 텍스트 → 정수 시퀀스 변환")
print("   - 시퀀스 길이 통일 (패딩/자르기)")
print()
print("⚡ 6단계: 성능 최적화")
print("   - 캐싱: 반복 접근 시 메모리에서 빠른 로드")
print("   - 프리페칭: GPU 유휴 시간 최소화")
print("   - 병렬 처리: 멀티코어 CPU 활용")
print()
print("✅ 결과: 딥러닝 모델 훈련 준비 완료!")
print(f"   - 입력 형태: (배치크기={BATCH_SIZE}, 시퀀스길이={OUTPUT_SEQUENCE_LENGTH})")
print(f"   - 어휘 사전 크기: {MAX_TOKENS:,}개")
print("   - 라벨: 0(부정), 1(긍정)")

## 10. 다음 단계 안내

이제 전처리된 데이터셋을 사용하여 딥러닝 모델을 구축할 수 있습니다.

### 모델 구축 예시 코드:

```python
# 간단한 감정 분석 모델 예시
model = keras.Sequential([
    keras.layers.Embedding(MAX_TOKENS, 128),
    keras.layers.LSTM(64, dropout=0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 모델 훈련
model.fit(
    train_ds_final,
    validation_data=val_ds_final,
    epochs=5
)
```

### 사용 가능한 데이터셋:
- `train_ds_final`: 훈련용 데이터셋
- `val_ds_final`: 검증용 데이터셋  
- `test_ds_final`: 테스트용 데이터셋

모든 데이터셋은 벡터화되어 있으며 성능 최적화가 적용되어 있습니다.